# Pré-Processamento de Endereços

## 1.Setup

Setup do ambiente para importar bibliotecas e dados.

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Carregando os dados
dados_geral = pd.read_csv('/content/drive/MyDrive/Dados de Consumo/Consumo 2024 - Cem Linhas Teste.csv')

In [6]:
# Visualizando as primeiras linhas dos dados
print(dados_geral.head())

   Unnamed: 0  EMP_CODIGO  REFERENCIA  COD_GRUPO  COD_SETOR_COMERCIAL  \
0           0           2  2024-05-01          4                   17   
1           1           2  2024-06-01          4                   87   
2           2           2  2024-07-01         14                   85   
3           3           2  2024-03-01          8                    5   
4           4           2  2024-01-01         12                   88   

   NUM_QUADRA  COD_ROTA_LEITURA  MATRICULA  SEQ_RESPONSAVEL  ECO_RESIDENCIAL  \
0          16                11   17188010           581756                0   
1          60                38   17424465           188809                1   
2         314                 9   17857770           454315                1   
3         112                25   17702922           311133                0   
4         284                43   17719490           735648                1   

   ...                 DSC_SIMULTANEA  VOLUME_ESTIMADO  VOLUME_ESTIMADO_ACUM  \


## 2.Conversão De Endereços

O seguinte script tem como fim converter coordenadas em bairros via a bibloteca do geopy. A função personalizada extrai o bairro e adiciona essa informação ao DataFrame.


In [7]:
!pip install geopy

In [8]:
!pip install geopy

from geopy.geocoders import Nominatim

# Inicializando o geolocator
geolocator = Nominatim(user_agent="data_user")

# Função para obter bairro a partir de latitude e longitude
def obter_bairro(row):
    coordenadas = f"{row['COD_LATITUDE']}, {row['COD_LONGITUDE']}"
    localizacao = geolocator.reverse(coordenadas, exactly_one=True, language='pt-BR')
    if localizacao:
        endereco = localizacao.raw['address']
        bairro = endereco.get('suburb', 'Não encontrado')
    else:
        bairro = 'Não encontrado'
    return bairro

# Aplicando a função para criar a nova coluna
dados_geral['BAIRRO'] = dados_geral.apply(obter_bairro, axis=1)

# Visualizando o resultado
print(dados_geral.head())

   Unnamed: 0  EMP_CODIGO  REFERENCIA  COD_GRUPO  COD_SETOR_COMERCIAL  \
0           0           2  2024-05-01          4                   17   
1           1           2  2024-06-01          4                   87   
2           2           2  2024-07-01         14                   85   
3           3           2  2024-03-01          8                    5   
4           4           2  2024-01-01         12                   88   

   NUM_QUADRA  COD_ROTA_LEITURA  MATRICULA  SEQ_RESPONSAVEL  ECO_RESIDENCIAL  \
0          16                11   17188010           581756                0   
1          60                38   17424465           188809                1   
2         314                 9   17857770           454315                1   
3         112                25   17702922           311133                0   
4         284                43   17719490           735648                1   

   ...  VOLUME_ESTIMADO  VOLUME_ESTIMADO_ACUM  FATURADO_MEDIA  \
0  ...         

In [9]:
#import pandas as pd

#pd.set_option('display.max_rows', None)
#dados_geral

## 3. Inferir bairros e coluna de bairros

O dicionário bairro_para_regiao mapeia bairros para suas respectivas regiões urbanas de Campo Grande, facilitando a associação automática de bairros a grandes áreas urbanas predefinidas.

In [10]:
# Dicionário de correspondência entre bairros e regiões urbanas
bairro_para_regiao = {
    # Imbirussú
    'Núcleo Industrial': 'Imbirussú',
    'Nova Campo Grande': 'Imbirussú',
    'Popular': 'Imbirussú',
    'Panamá': 'Imbirussú',
    'Santo Antônio': 'Imbirussú',
    'Santo Amaro': 'Imbirussú',
    'Sobrinho': 'Imbirussú',

    # Lagoa
    'Caiobá': 'Lagoa',
    'Tarumã': 'Lagoa',
    'Coophavillas': 'Lagoa',
    'Batistão': 'Lagoa',
    'Tijuca': 'Lagoa',
    'Leblon': 'Lagoa',
    'São Conrado': 'Lagoa',
    'União': 'Lagoa',
    'Caiçara': 'Lagoa',
    'Taveirópolis': 'Lagoa',
    'Bandeirantes': 'Lagoa',

    # Anhanduizinho
    'Centro Oeste': 'Anhanduizinho',
    'Los Angeles': 'Anhanduizinho',
    'Lageado': 'Anhanduizinho',
    'Centenário': 'Anhanduizinho',
    'Alves Pereira': 'Anhanduizinho',
    'Pioneiros': 'Anhanduizinho',
    'Aero Rancho': 'Anhanduizinho',
    'Parati': 'Anhanduizinho',
    'Guanandi': 'Anhanduizinho',
    'Piratininga': 'Anhanduizinho',
    'Jacy': 'Anhanduizinho',
    'Jockey Clube': 'Anhanduizinho',
    'Jardim América': 'Anhanduizinho',
    'América': 'Anhanduizinho',

    # Bandeira
    'Moreninha': 'Bandeira',
    'Universitário': 'Bandeira',
    'Rita Vieira': 'Bandeira',
    'Jardim Paulista': 'Bandeira',
    'TV Morena': 'Bandeira',
    'Vilasboas': 'Bandeira',
    'São Lourenço': 'Bandeira',
    'Tiradentes': 'Bandeira',
    'Maria Aparecida Pedrossian': 'Bandeira',
    'Carlota': 'Bandeira',
    'Dr. Albuquerque': 'Bandeira',

    # Centro
    'Centro': 'Centro',
    'São Francisco': 'Centro',
    'Cruzeiro': 'Centro',
    'Jardim dos Estados': 'Centro',
    'Bela Vista': 'Centro',
    'Itanhangá': 'Centro',
    'São Bento': 'Centro',
    'Monte Líbano': 'Centro',
    'Glória': 'Centro',
    'Carvalho': 'Centro',
    'Amambaí': 'Centro',
    'Cabreúva': 'Centro',
    'Planalto': 'Centro',

    # Segredo
    'José Abrão': 'Segredo',
    'Nasser': 'Segredo',
    'Seminário': 'Segredo',
    'Monte Castelo': 'Segredo',
    'Mata do Segredo': 'Segredo',
    'Coronel Antonino': 'Segredo',
    'Nova Lima': 'Segredo',

    # Prosa
    'Autonomista': 'Prosa',
    'Santa Fé': 'Prosa',
    'Chácara Cachoeira': 'Prosa',
    'Carandá': 'Prosa',
    'Margarida': 'Prosa',
    'Mata do Jacinto': 'Prosa',
    'Novos Estados': 'Prosa',
    'Estrela Dalva': 'Prosa',
    'Veraneio': 'Prosa',
    'Chácara dos Poderes': 'Prosa',
    'Noroeste': 'Prosa'
}


A função abaixo recebe uma linha do DataFrame, extrai o bairro e retorna a região urbana correspondente a partir do dicionário.

In [11]:
# Função para obter a região urbana a partir do bairro
def obter_regiao_urbana(row):
    bairro = row['BAIRRO']
    return bairro_para_regiao.get(bairro, 'Não encontrado')

# Aplicando a função para criar a nova coluna REGIAO URBANA
dados_geral['REGIAO URBANA'] = dados_geral.apply(obter_regiao_urbana, axis=1)

# Visualizando o resultado
print(dados_geral.head())

   Unnamed: 0  EMP_CODIGO  REFERENCIA  COD_GRUPO  COD_SETOR_COMERCIAL  \
0           0           2  2024-05-01          4                   17   
1           1           2  2024-06-01          4                   87   
2           2           2  2024-07-01         14                   85   
3           3           2  2024-03-01          8                    5   
4           4           2  2024-01-01         12                   88   

   NUM_QUADRA  COD_ROTA_LEITURA  MATRICULA  SEQ_RESPONSAVEL  ECO_RESIDENCIAL  \
0          16                11   17188010           581756                0   
1          60                38   17424465           188809                1   
2         314                 9   17857770           454315                1   
3         112                25   17702922           311133                0   
4         284                43   17719490           735648                1   

   ...  VOLUME_ESTIMADO_ACUM  FATURADO_MEDIA  COD_LEITURA_INT  STA_TROCA  \
0  .